In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import csv
import os
import imageio
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import statistics
import ipywidgets as widgets
from ipywidgets import interactive, interact_manual
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
pyo.init_notebook_mode()
%matplotlib inline

In [ ]:
#params
xgrd = 100
ygrd = 100
numvals = 3
data = []
zvals = []
mean = []

In [ ]:
# make values from -5 to 5, for this example
def gengrid(x,y):
    global zvals
    zvals = np.random.randint(numvals,size=(x,y))


def showgrid(zvals,xgrd,ygrd):
    # make a color map of fixed colors
    cmap = mpl.colors.ListedColormap(['yellow','black','orange'])
    bounds=[0,1,2,3]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)


    # tell imshow about color map so that only set colors are used
    img = plt.imshow(zvals,interpolation='nearest',
                        cmap = cmap,norm=norm)
    # make a color bar
    plt.colorbar(img,boundaries=bounds,ticks=[0,1,2])
    
    return img

def extratranddata(data, iterations, nsamples):
    data = {'y':[]} #create dataframe
    df = pd.DataFrame(data)
    
    for n in range(0,nsamples):
        xlist = []
        ylist = []
        for l in range(0,iterations+1):
            y = np.random.choice( zvals.ravel(),1,replace=False)
            x = l
            xlist.append(x)
            ylist.append(y)
            # write the actual content line by line
        #print(ylist)
        df['y'] = ylist
        df['SMA' + str(n)] = df.y.rolling(iterations, min_periods=1).mean()
        del df['y']
    return df

#input must be a 2d numpy array.
def samplerev_spag(inputdata, nsamples, iterations):
    global data
    global mean
    
    
    #plot mean
    mean = np.mean(zvals)
    #plt.axhline(mean, color='r', linestyle=':') #supressed mean
    #plt.figtext(1, 0, mean, wrap=True, horizontalalignment='right', fontsize=12)
    
    plt.plot(inputdata)
    #plt.scatter(xlist,ylist,color='r')
    plt.xlim([0,nsamples])
    plt.ylim([0,numvals-1])
    #plt.xlabel('Iterations')
    plt.ylabel('Measured value')

def plot(data,gridsize,nsamples,iterations):
    samplerev_test(zvals,500,50)
    plotgrid()

def max(df):
    df['Max'] = df.max(axis=1)

def min(df):
    df['Min'] = df.min(axis=1)

def minmaxplot(inputdata, iterations, smooth):
    max(inputdata)
    min(inputdata)

    max1 = inputdata['Max']
    min1 = inputdata['Min']

    if smooth == True:
        ymax = max1.values.tolist()

        ymin = min1.values.tolist()

        #definex
        x = []
        for i in range (0,iterations+1):
            x.append(i)

        #curve is y = 1/x
        smax = UnivariateSpline(x, ymax, k=5, s=iterations)
        xsmax = np.linspace(0, iterations , 100)
        ysmax = smax(xsmax)

        smin = UnivariateSpline(x, ymin, k=5, s=iterations)
        xsmin = np.linspace(0, iterations , 100)
        ysmin = smin(xsmin)

        plt.plot(xsmax, ysmax,color='red')
        plt.plot(xsmin, ysmin,color='blue')
    else:
        return

    plt.plot(max1, 'r+', alpha=0.3)
    plt.plot(min1, 'b+', alpha=0.3)
    plt.axhline(mean, color='r', linestyle=':')#mean line
    plt.figtext(1, 0, mean, wrap=True, horizontalalignment='right', fontsize=12)#mean text
    plt.xlim([0,iterations])
    plt.ylim([0,numvals-1])
    plt.xlabel('Iterations')
    plt.ylabel('Measured value')


def curvefit(inputdata, maxvar, minvar,  iterations): #variable must be string, fits curve y = 1/x 
    max(inputdata)
    min(inputdata)

    ymax = inputdata[maxvar]
    ymax = ymax.values.tolist()

    ymin = inputdata[minvar]
    ymin = ymin.values.tolist()

    #definex
    x = []
    for i in range (0,iterations+1):
        x.append(i)

    #curve is y = 1/x
    smax = UnivariateSpline(x, ymax, k=5, s=iterations)
    xsmax = np.linspace(0, iterations*1.2 , 100)
    ysmax = smax(xsmax)

    smin = UnivariateSpline(x, ymin, k=5, s=iterations)
    xsmin = np.linspace(0, iterations*1.2 , 100)
    ysmin = smin(xsmin)

    plt.plot(xsmax, ysmax)
    plt.plot(xsmin, ysmin)

    plt.axhline(mean, color='r', linestyle=':')#mean line
    plt.figtext(1, 0, (print("Mean:"), mean), wrap=True, horizontalalignment='right', fontsize=12)#mean text
    plt.xlim([0,iterations])
    plt.ylim([0,numvals-1])
    plt.xlabel('Iterations')
    plt.ylabel('Measured value')


    




When we sample this grid many times to produce many interations and plot them, they trend towards the mean, the REV is the number of iterations it takes for the mean average variability to be statistically insignificant

In [ ]:
def generation(xgrd):
    ygrd = xgrd
    global zvals
    gengrid(xgrd,ygrd)
    print("We can produce a random grid size " ,xgrd, "x ", ygrd, " with ", numvals, " properties:")
    showgrid(zvals,xgrd,ygrd)
    plt.show(block=False)


grid_generation = interactive(generation, xgrd =(50,100,10))
grid_generation

In [ ]:
def series(iterations, samples):
    inputdata = extratranddata(zvals, iterations, samples)
    plt.show(samplerev_spag(inputdata,iterations, samples))
    plt.show(minmaxplot(inputdata, iterations, True))
    #plt.show(curvefit(inputdata, 'Max', 'Min', iterations)) #depreciated
    return()


interactive_plot = interactive(series, iterations =(50,1000,50), samples =(10,50,5))
interactive_plot